In [1]:
# change directory
import os
rootdir = os.path.dirname(os.getcwd())
os.chdir(rootdir)

# import third libs for data
from datetime import datetime as dt
from datetime import timedelta as td
import numpy as np
import pandas as pd

# import config
from config import DATA_DIR

# Chargement des donnees

In [2]:
benchmarkname = 'HGD_dialysis_2018_2021'
benchmarkdir = os.path.join(DATA_DIR, 'raw', benchmarkname)
benchmarkdir

'D:\\Recherches\\Travaux\\RCEMP\\rcemp-dial\\data\\raw\\HGD_dialysis_2018_2021'

## chargement des seances

In [3]:
datapath = os.path.join(benchmarkdir, 'donnees_seances.csv')
seances = pd.read_csv(datapath, sep=';', index_col='Date', parse_dates=True, dayfirst=True)
seances.head()

,ID,Tour,Patients,Debut,Duree,jour
Date,,,,,,
2018-03-26,1,Tour1,21,5,5,0
2018-03-26,2,Tour2,21,10,5,0
2018-03-26,3,Tour3,21,15,5,0
2018-03-26,4,Tour4,8,20,5,0
2018-03-27,5,Tour1,21,5,5,1


In [4]:
seances.dtypes

ID           int64
Tour        object
Patients     int64
Debut        int64
Duree        int64
jour         int64
dtype: object

## chargement des generateurs

In [5]:
datapath = os.path.join(benchmarkdir, 'donnees_generateurs.csv')
generateurs = pd.read_csv(datapath, sep=';', index_col='Numero')
generateurs.head()

,Code,TT,TO,TNR,TR,TAP,TF,TE,TN,TU,TRE,TRSi
Numero,,,,,,,,,,,,
1,7SXAWU98,30576,22932,932,22000,6301.08,15698.92,4.92,15694,15694,0,NaN
2,7SXAWT97,30576,22932,932,22000,19262.22,2737.78,1.78,2736,2736,0,NaN
3,7SXAWU81,30576,22932,932,22000,9462.73,12537.27,1.27,12536,12536,0,NaN
4,7SXAWU91,30576,22932,932,22000,7857.71,14142.29,2.29,14140,14140,0,NaN
5,7SXAWS81,30576,22932,932,22000,5639.52,16360.48,4.48,16356,16356,0,NaN


In [6]:
generateurs.dtypes

Code     object
TT        int64
TO        int64
TNR       int64
TR        int64
TAP     float64
TF      float64
TE      float64
TN        int64
TU        int64
TRE       int64
TRSi    float64
dtype: object

In [7]:
generateurs.Code.sort_values()

Numero
24    6SX4LV10
21    7SXAWS80
5     7SXAWS81
20    7SXAWS82
2     7SXAWT97
8     7SXAWT98
14    7SXAWU79
17    7SXAWU80
3     7SXAWU81
12    7SXAWU82
22    7SXAWU83
18    7SXAWU84
19    7SXAWU85
9     7SXAWU86
7     7SXAWU87
11    7SXAWU89
16    7SXAWU90
4     7SXAWU91
15    7SXAWU93
13    7SXAWU94
23    7SXAWU95
6     7SXAWU96
10    7SXAWU97
1     7SXAWU98
Name: Code, dtype: object

## chargements de l'historique des pannes

In [8]:
datapath = os.path.join(benchmarkdir, 'donnees_pannes.csv')
pannes = pd.read_csv(datapath, sep=';', index_col='Machines')
pannes = pannes.transpose()
pannes.index = pd.to_datetime(pannes.index, format='%d/%m/%Y')
pannes.fillna(0, inplace=True)
pannes = pannes.astype('Int64')
pannes.head()


Machines,7SXAWU98,7SXAWT97,7SXAWU81,7SXAWU90,7SXAWS81,7SXAWU96,7SXAWU87,7SXAWT98,7SXAWU86,7SXAWU97,...,7SXAWU93,7SXAWU91,7SXAWU80,7SXAWU84,7SXAWU85,7SXAWS82,7SXAWS80,7SXAWU83,7SXAWU95,6SX4LV10
2018-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
pannes.sum()

Machines
7SXAWU98     0
7SXAWT97     0
7SXAWU81    12
7SXAWU90     6
7SXAWS81    15
7SXAWU96    12
7SXAWU87     6
7SXAWT98     6
7SXAWU86     0
7SXAWU97    18
7SXAWU89     3
7SXAWU82     3
7SXAWU94    12
7SXAWU79     9
7SXAWU93     3
7SXAWU91    12
7SXAWU80    18
7SXAWU84     6
7SXAWU85     6
7SXAWS82     6
7SXAWS80     9
7SXAWU83     9
7SXAWU95    12
6SX4LV10    12
dtype: int64

## chargement des tranches horaires

In [10]:
datapath = os.path.join(benchmarkdir, 'tranches_maintenances.csv')
tranches_maintenances = pd.read_csv(datapath)
tranches_maintenances.set_index('Period', inplace=True)
tranches_maintenances

,D1,D2,D3,D4,D5,D6,D7
Period,,,,,,,
0H-1H,--,--,--,--,--,--,--
1H-2H,--,--,--,--,--,--,--
2H-3H,--,--,--,--,--,--,--
3H-4H,service,service,service,service,service,service,--
4H-5H,service,service,service,service,service,service,--
5H-6H,service,service,service,service,service,service,--
6H-7H,service,service,service,service,service,service,--
7H-8H,service,service,service,service,service,service,--
8H-9H,service,service,service,service,service,service,--


In [11]:
datapath = os.path.join(benchmarkdir, 'tranches_soins.csv')
tranches_soins = pd.read_csv(datapath)
tranches_soins.set_index('Period', inplace=True)
tranches_soins

,D1,D2,D3,D4,D5,D6,D7
Period,,,,,,,
0H-1H,--,soins4,soins4,soins4,soins4,soins4,soins4
1H-2H,--,--,--,--,--,--,--
2H-3H,--,--,--,--,--,--,--
3H-4H,--,--,--,--,--,--,--
4H-5H,--,--,--,--,--,--,--
5H-6H,soins1,soins1,soins1,soins1,soins1,soins1,--
6H-7H,soins1,soins1,soins1,soins1,soins1,soins1,--
7H-8H,soins1,soins1,soins1,soins1,soins1,soins1,--
8H-9H,soins1,soins1,soins1,soins1,soins1,soins1,--


# Identification de la periode d'echantillonage

## periodes de collecte des seances

In [12]:
debut_seances = seances['Patients'].index.min()
fin_seances = seances['Patients'].index.max()
fin_seances= fin_seances + pd.DateOffset(1)
debut_seances, fin_seances

(Timestamp('2018-03-26 00:00:00'), Timestamp('2021-05-23 00:00:00'))

## periodes de collecte des pannes

In [13]:
debut_pannes = []
fin_pannes = []
for machine in pannes.columns:
    test = pannes[machine]
    periodes = test[test == 1].index
    if len(periodes) == 0:
        continue
    debut_machine = periodes.min()
    debut_pannes.append(debut_machine)
    fin_machine = periodes.max()
    fin_pannes.append(fin_machine)
    print(machine, debut_machine, fin_machine)
debut_pannes = max(debut_pannes)
fin_pannes = max(fin_pannes)
fin_pannes= fin_pannes + pd.DateOffset(1)
debut_pannes, fin_pannes

7SXAWU81 2020-09-25 00:00:00 2021-06-09 00:00:00
7SXAWU90 2020-10-07 00:00:00 2020-10-16 00:00:00
7SXAWS81 2020-05-03 00:00:00 2021-07-06 00:00:00
7SXAWU96 2020-07-15 00:00:00 2021-06-05 00:00:00
7SXAWU87 2020-08-19 00:00:00 2021-05-01 00:00:00
7SXAWT98 2020-11-12 00:00:00 2021-05-12 00:00:00
7SXAWU97 2020-07-10 00:00:00 2021-02-18 00:00:00
7SXAWU89 2020-09-23 00:00:00 2020-09-25 00:00:00
7SXAWU82 2020-09-23 00:00:00 2020-09-25 00:00:00
7SXAWU94 2020-09-25 00:00:00 2021-12-08 00:00:00
7SXAWU79 2020-06-23 00:00:00 2021-05-12 00:00:00
7SXAWU93 2020-10-15 00:00:00 2020-10-17 00:00:00
7SXAWU91 2020-09-18 00:00:00 2021-07-06 00:00:00
7SXAWU80 2020-09-25 00:00:00 2021-05-12 00:00:00
7SXAWU84 2020-02-20 00:00:00 2020-07-31 00:00:00
7SXAWU85 2020-08-29 00:00:00 2021-06-23 00:00:00
7SXAWS82 2020-09-23 00:00:00 2021-05-05 00:00:00
7SXAWS80 2020-07-19 00:00:00 2020-12-25 00:00:00
7SXAWU83 2020-09-23 00:00:00 2020-12-09 00:00:00
7SXAWU95 2020-04-05 00:00:00 2021-04-02 00:00:00
6SX4LV10 2020-09-23 

(Timestamp('2020-11-12 00:00:00'), Timestamp('2021-12-09 00:00:00'))

# Identification des patients

## Regroupement des patients par jour

In [14]:
seances_by_date = seances.groupby('Date')
seances_sum_by_date = seances_by_date.sum(numeric_only=True)
patients_count = seances_sum_by_date['Patients']
patients = pd.DataFrame(patients_count)
patients.columns = ['number']
patients.head()


,number
Date,
2018-03-26,71
2018-03-27,72
2018-03-28,68
2018-03-29,67
2018-03-30,72


In [15]:
patients['day'] = patients_count.index.weekday + 1
patients.head()

,number,day
Date,,
2018-03-26,71,1
2018-03-27,72,2
2018-03-28,68,3
2018-03-29,67,4
2018-03-30,72,5


## Calcul des preferences des patients

In [16]:
seances_by_tour = seances.groupby('Tour')
seances_by_tour_min = seances_by_tour.min()
seances_by_tour_max = seances_by_tour.max()
seances_by_tour_min.head()
patients['release_date'] = seances_by_tour_min.loc['Tour1', 'Debut']
patients['due_date'] = seances_by_tour_max.loc['Tour4', 'Debut']
patients['duration'] = seances_by_tour_min.loc['Tour1', 'Duree']
patients.head()

,number,day,release_date,due_date,duration
Date,,,,,
2018-03-26,71,1,5,20,5
2018-03-27,72,2,5,20,5
2018-03-28,68,3,5,20,5
2018-03-29,67,4,5,20,5
2018-03-30,72,5,5,20,5


## Identification des periodes de soins

In [17]:
patients.reset_index(inplace=True)
patients['period'] = patients['Date'].dt.to_period('W').apply(lambda r: r.end_time.date)
patients['period'] = pd.to_datetime(patients['period'])
patients.set_index('Date', inplace=True)
patients.head(10)

,number,day,release_date,due_date,duration,period
Date,,,,,,
2018-03-26,71,1,5,20,5,2018-04-01
2018-03-27,72,2,5,20,5,2018-04-01
2018-03-28,68,3,5,20,5,2018-04-01
2018-03-29,67,4,5,20,5,2018-04-01
2018-03-30,72,5,5,20,5,2018-04-01
2018-03-31,60,6,5,20,5,2018-04-01
2018-04-02,76,1,5,20,5,2018-04-08
2018-04-03,67,2,5,20,5,2018-04-08
2018-04-04,62,3,5,20,5,2018-04-08


# Identification des machines

## Identification des machines par defaut

In [18]:
total_pannes = pannes.sum().astype('Int64')
machines_utiles = total_pannes[total_pannes > 0].index.to_list()
machines_utiles, len(machines_utiles)

(['7SXAWU81',
  '7SXAWU90',
  '7SXAWS81',
  '7SXAWU96',
  '7SXAWU87',
  '7SXAWT98',
  '7SXAWU97',
  '7SXAWU89',
  '7SXAWU82',
  '7SXAWU94',
  '7SXAWU79',
  '7SXAWU93',
  '7SXAWU91',
  '7SXAWU80',
  '7SXAWU84',
  '7SXAWU85',
  '7SXAWS82',
  '7SXAWS80',
  '7SXAWU83',
  '7SXAWU95',
  '6SX4LV10'],
 21)

## identification des machines de secours

In [19]:
machines_secours = total_pannes[total_pannes == 0].index
machines_secours

Index(['7SXAWU98', '7SXAWT97', '7SXAWU86'], dtype='object', name='Machines')

In [20]:
temps_secours = generateurs.set_index('Code')
temps_secours = temps_secours.loc[machines_secours, 'TF']
temps_secours.sort_values(ascending=False)

Machines
7SXAWU98    15698.92
7SXAWT97     2737.78
7SXAWU86     1705.91
Name: TF, dtype: float64

In [21]:
ordre_secours = temps_secours.sort_values(ascending=False).index
ordre_secours = ordre_secours.to_list()
ordre_secours

['7SXAWU98', '7SXAWT97', '7SXAWU86']

## identification des machines utilisees

In [22]:
use_jours = pd.DataFrame(0, index=patients.index, columns=pannes.columns)
nb_machines = len(machines_utiles)
for jour in pannes.index:
    test = pannes.loc[jour]
    machines_pannes = test[test == 1].index
    machines_pannes = machines_pannes.to_list()
    if len(machines_pannes) > 0:
        machines_jour= [m for m in machines_utiles if m not in machines_pannes]
        machines_jour.extend(ordre_secours[:nb_machines - len(machines_jour)])
    else:
        machines_jour = list(machines_utiles)
    use_jours.loc[jour, machines_jour] = 1
    # print(jour, len(machines_jour), machines_pannes)
#     ids = machines_jour
#     print(jour, [(ids.count(i),i) for i in ids], len(ids))
use_jours.shape

(1471, 24)

In [23]:
machine1 = machines_utiles[0]
use_machine1 = use_jours[machine1]
use_machine1[use_machine1 > 0 ].head()

Date
2018-03-26    1.0
2018-03-27    1.0
2018-03-28    1.0
2018-03-29    1.0
2018-03-30    1.0
Name: 7SXAWU81, dtype: float64

In [24]:
machine2 = machines_secours[0]
use_machine2 = use_jours[machine2]
use_machine2[use_machine2 == 0 ].head()

Date
2018-03-26    0.0
2018-03-27    0.0
2018-03-28    0.0
2018-03-29    0.0
2018-03-30    0.0
Name: 7SXAWU98, dtype: float64

In [25]:
total_use_jours = use_jours.sum(axis=1)
total_use_jours.head()

Date
2018-03-26    21.0
2018-03-27    21.0
2018-03-28    21.0
2018-03-29    21.0
2018-03-30    21.0
dtype: float64

In [26]:
total_use_jours[total_use_jours < nb_machines].head()

Date
2020-09-23    19.0
2020-09-24    19.0
2020-09-25    16.0
2020-12-07    20.0
2020-12-08    19.0
dtype: float64

## calcul des temps d'utilisation journaliere

In [27]:
duree_traitement = 4
nb_machines = len(machines_utiles)
temps = seances['Patients'] * duree_traitement
temps.name = 'use_duration'
temps = temps.to_frame()
temps.head()

,use_duration
Date,
2018-03-26,84
2018-03-26,84
2018-03-26,84
2018-03-26,32
2018-03-27,84


In [28]:
temps_jour1 = temps.resample('D').sum()
temps_jour1.head()

,use_duration
Date,
2018-03-26,284
2018-03-27,288
2018-03-28,272
2018-03-29,268
2018-03-30,288


In [29]:
temps_jour1.tail()

,use_duration
Date,
2021-05-18,272
2021-05-19,260
2021-05-20,252
2021-05-21,284
2021-05-22,240


In [30]:
print(temps_jour1.index)

DatetimeIndex(['2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29',
               '2018-03-30', '2018-03-31', '2018-04-01', '2018-04-02',
               '2018-04-03', '2018-04-04',
               ...
               '2021-05-13', '2021-05-14', '2021-05-15', '2021-05-16',
               '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20',
               '2021-05-21', '2021-05-22'],
              dtype='datetime64[ns]', name='Date', length=1154, freq='D')


In [31]:
# test = use_jours.sum(axis=1)
# for i in test.index:
#     print(i, test.loc[i])

In [32]:
temps_jour2 = pd.DataFrame(0, index=temps_jour1.index, columns=pannes.columns)
for jour in temps_jour1.index:
# for jour in temps_jour1.index[:10]:
# for jour in use_jours.index:
    try:
        test = use_jours.loc[jour]
    except KeyError:
        print(jour, 'ignored')
        continue
    machines_jour = test[test == 1].index
    machines_jour = machines_jour.to_list()
    nb_machines_jour = len(machines_jour)
    if nb_machines_jour == 0:
        print(jour, 'no machines')
        continue
    
    nb_total = temps_jour1.loc[jour, 'use_duration']
    nb_seances = nb_total // nb_machines_jour
    nb_retards = nb_total % nb_machines_jour
    if nb_machines_jour < nb_machines:
        print(jour, nb_seances, nb_retards, len(machines_jour))

    temps_jour2.loc[jour, machines_jour] = nb_seances * duree_traitement
    # print(jour, nb_seances * duree_traitement, temps_jour2.loc[jour, machines_jour])
    for i in range(nb_retards):
        machine = machines_jour[i]
        temps_jour2.loc[jour, machine] += duree_traitement

temps_jour2.shape

2020-09-23 00:00:00 11 11 19
2020-09-24 00:00:00 13 17 19
2020-09-25 00:00:00 16 4 16
2020-12-07 00:00:00 13 12 20
2020-12-08 00:00:00 15 11 19
2021-05-10 00:00:00 14 18 19
2021-05-11 00:00:00 14 14 19
2021-05-12 00:00:00 14 14 19


(1154, 24)

In [33]:
temps_jour2.sum(axis=1).head(25)

Date
2018-03-26    1136
2018-03-27    1152
2018-03-28    1088
2018-03-29    1072
2018-03-30    1152
2018-03-31     960
2018-04-01       0
2018-04-02    1216
2018-04-03    1072
2018-04-04     992
2018-04-05    1104
2018-04-06    1152
2018-04-07     992
2018-04-08       0
2018-04-09    1264
2018-04-10    1104
2018-04-11    1008
2018-04-12    1216
2018-04-13    1072
2018-04-14     992
2018-04-15       0
2018-04-16    1184
2018-04-17    1200
2018-04-18     960
2018-04-19    1152
Freq: D, dtype: int64

In [34]:
temps_jour2.sum(axis=1).tail(25)

Date
2021-04-28    1184
2021-04-29    1008
2021-04-30    1072
2021-05-01     960
2021-05-02       0
2021-05-03    1120
2021-05-04    1088
2021-05-05    1072
2021-05-06    1008
2021-05-07    1072
2021-05-08    1008
2021-05-09       0
2021-05-10    1136
2021-05-11    1120
2021-05-12    1120
2021-05-13    1008
2021-05-14    1056
2021-05-15    1008
2021-05-16       0
2021-05-17    1136
2021-05-18    1088
2021-05-19    1040
2021-05-20    1008
2021-05-21    1136
2021-05-22     960
Freq: D, dtype: int64

In [35]:
temps_jour2.tail(25)

Machines,7SXAWU98,7SXAWT97,7SXAWU81,7SXAWU90,7SXAWS81,7SXAWU96,7SXAWU87,7SXAWT98,7SXAWU86,7SXAWU97,...,7SXAWU93,7SXAWU91,7SXAWU80,7SXAWU84,7SXAWU85,7SXAWS82,7SXAWS80,7SXAWU83,7SXAWU95,6SX4LV10
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-28,0,0,60,60,56,56,56,56,0,56,...,56,56,56,56,56,56,56,56,56,56
2021-04-29,48,48,0,48,48,48,0,48,48,48,...,48,48,0,48,48,48,48,48,48,48
2021-04-30,52,52,0,52,52,52,0,52,52,52,...,52,52,0,52,52,48,48,48,48,48
2021-05-01,48,48,0,48,48,48,0,48,48,48,...,44,44,0,44,44,44,44,44,44,44
2021-05-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-05-03,56,56,56,56,56,56,56,52,0,52,...,52,0,52,52,52,0,52,52,52,52
2021-05-04,52,52,52,52,52,52,52,52,0,52,...,52,0,52,52,52,0,52,52,52,48
2021-05-05,52,52,52,52,52,52,52,52,0,52,...,52,0,52,52,48,0,48,48,48,48
2021-05-06,0,0,48,48,48,48,48,48,0,48,...,48,48,48,48,48,48,48,48,48,48


## calcul des temps initiaux et finaux

In [36]:
temps_initiaux = pd.DataFrame(0, index=temps_jour2.index, columns=temps_jour2.columns)
temps_finaux = pd.DataFrame(0, index=temps_jour2.index, columns=temps_jour2.columns)
for machine in temps_finaux.columns:
    print('\n', machine)
    t0 = 0
    t1 = 0
    test = pannes[machine]
    for jour in temps_finaux.index:
        if test.loc[jour] == 1:
            print('pannes', jour, machine, t0, t1)
            t0 = 0
        t1 = t0 + temps_jour2.loc[jour, machine]
        temps_initiaux.loc[jour, machine] = t0
        temps_finaux.loc[jour, machine] = t1
        # print(jour, t0, t1)
        t0 = t1
    


 7SXAWU98

 7SXAWT97

 7SXAWU81
pannes 2020-09-25 00:00:00 7SXAWU81 41440 41440
pannes 2020-09-26 00:00:00 7SXAWU81 0 0
pannes 2020-09-27 00:00:00 7SXAWU81 0 0
pannes 2021-04-29 00:00:00 7SXAWU81 9864 9864
pannes 2021-04-30 00:00:00 7SXAWU81 0 0
pannes 2021-05-01 00:00:00 7SXAWU81 0 0
pannes 2021-05-10 00:00:00 7SXAWU81 308 308
pannes 2021-05-11 00:00:00 7SXAWU81 0 0
pannes 2021-05-12 00:00:00 7SXAWU81 0 0

 7SXAWU90
pannes 2020-10-07 00:00:00 7SXAWU90 41932 41932
pannes 2020-10-08 00:00:00 7SXAWU90 0 0
pannes 2020-10-09 00:00:00 7SXAWU90 0 0
pannes 2020-10-14 00:00:00 7SXAWU90 148 148
pannes 2020-10-15 00:00:00 7SXAWU90 0 0
pannes 2020-10-16 00:00:00 7SXAWU90 0 0

 7SXAWS81
pannes 2020-05-03 00:00:00 7SXAWS81 34956 34956
pannes 2020-05-04 00:00:00 7SXAWS81 0 0
pannes 2020-05-05 00:00:00 7SXAWS81 0 0
pannes 2021-02-07 00:00:00 7SXAWS81 12352 12352
pannes 2021-02-08 00:00:00 7SXAWS81 0 0
pannes 2021-02-09 00:00:00 7SXAWS81 0 0
pannes 2021-03-31 00:00:00 7SXAWS81 2312 2312
pannes 2021-0

In [37]:
temps_initiaux[machines_utiles[:8]].head()

Machines,7SXAWU81,7SXAWU90,7SXAWS81,7SXAWU96,7SXAWU87,7SXAWT98,7SXAWU97,7SXAWU89
Date,,,,,,,,
2018-03-26,0,0,0,0,0,0,0,0
2018-03-27,56,56,56,56,56,56,56,56
2018-03-28,112,112,112,112,112,112,112,112
2018-03-29,164,164,164,164,164,164,164,164
2018-03-30,216,216,216,216,216,216,216,216


In [38]:
temps_initiaux[machines_secours].head()

Machines,7SXAWU98,7SXAWT97,7SXAWU86
Date,,,
2018-03-26,0,0,0
2018-03-27,0,0,0
2018-03-28,0,0,0
2018-03-29,0,0,0
2018-03-30,0,0,0


In [39]:
temps_finaux[machines_utiles[:8]].tail()

Machines,7SXAWU81,7SXAWU90,7SXAWS81,7SXAWU96,7SXAWU87,7SXAWT98,7SXAWU97,7SXAWU89
Date,,,,,,,,
2021-05-18,256,9872,1860,2180,744,256,4068,10552
2021-05-19,308,9924,1912,2232,796,308,4120,10604
2021-05-20,356,9972,1960,2280,844,356,4168,10652
2021-05-21,412,10028,2016,2336,900,412,4224,10708
2021-05-22,460,10076,2064,2384,948,460,4272,10756


In [40]:
temps_finaux[machines_secours].tail()

Machines,7SXAWU98,7SXAWT97,7SXAWU86
Date,,,
2021-05-18,4516,1896,880
2021-05-19,4516,1896,880
2021-05-20,4516,1896,880
2021-05-21,4516,1896,880
2021-05-22,4516,1896,880


In [41]:
machines = []
for code in temps_finaux.columns:
    for date in temps_finaux.index:
        machine = {"code":code, "date":date}
        machine['use_duration'] = temps_initiaux.loc[date, code]
        machine['final_duration'] = temps_finaux.loc[date, code]
        machines.append(machine)
machines = pd.DataFrame(machines)
machines['period'] = machines['date'].dt.to_period('W').apply(lambda r: r.end_time.date)
machines.head()

,code,date,use_duration,final_duration,period
0,7SXAWU98,2018-03-26,0,0,2018-04-01
1,7SXAWU98,2018-03-27,0,0,2018-04-01
2,7SXAWU98,2018-03-28,0,0,2018-04-01
3,7SXAWU98,2018-03-29,0,0,2018-04-01
4,7SXAWU98,2018-03-30,0,0,2018-04-01


In [42]:
machines.tail()

,code,date,use_duration,final_duration,period
27691,6SX4LV10,2021-05-18,956,1004,2021-05-23
27692,6SX4LV10,2021-05-19,1004,1052,2021-05-23
27693,6SX4LV10,2021-05-20,1052,1100,2021-05-23
27694,6SX4LV10,2021-05-21,1100,1152,2021-05-23
27695,6SX4LV10,2021-05-22,1152,1196,2021-05-23


# Echantillonage par phases

In [43]:
# fix random seed
np.random.seed(0)

## Echantillonage des semaines
ces echantillons sont utilises pour la comparaison entre ordonnancement manuel et intelligent

In [44]:
# debut_sample1 = max(debut_seances, debut_pannes)
debut_sample1 = debut_seances
debut_sample1

Timestamp('2018-03-26 00:00:00')

In [45]:
# fin_sample1 = min(fin_seances, fin_pannes)
fin_sample1 = fin_seances
fin_sample1

Timestamp('2021-05-23 00:00:00')

In [46]:
periode_sample1 = temps_finaux.mean(axis=1).resample('W').max()
periode_sample1.index.name = 'periode'
periode_sample1.name = 'duration'
periode_sample1 = periode_sample1.reset_index()
periode_sample1['annee'] = periode_sample1['periode'].dt.year
#periode_sample1 = periode_sample1[periode_sample1 < 15000]
# periode_sample1 = periode_sample1[periode_sample1 < 11000]
# periode_sample1 = periode_sample1[debut_sample1:fin_sample1]
periode_sample1 = periode_sample1[periode_sample1['annee'].isin([2018, 2020])]
periode_sample1 = periode_sample1[periode_sample1['duration'] < 7000]
periode_sample1

,periode,duration,annee
0,2018-04-01,273.333333,2018
1,2018-04-08,545.333333,2018
2,2018-04-15,822.666667,2018
3,2018-04-22,1094.666667,2018
4,2018-04-29,1366.666667,2018
5,2018-05-06,1629.333333,2018
6,2018-05-13,1897.333333,2018
7,2018-05-20,2166.000000,2018
8,2018-05-27,2432.000000,2018
9,2018-06-03,2697.333333,2018


In [47]:
# col1 = temps_finaux.columns[0]
# periode_sample1 = temps_finaux.loc[debut_sample1:fin_sample1, col1]
# periode_sample1  = periode_sample1.resample('W').sum()
# periode_sample1.index.name = 'periode'
# periode_sample1.name = '_'
# periode_sample1 = periode_sample1.reset_index().drop('_', axis=1)
# periode_sample1['annee'] = periode_sample1['periode'].dt.year
# periode_sample1.head()

In [48]:
periode_sample1.shape

(39, 3)

In [49]:
nb_sample1 = 12
periode_sample2 = []
for annee, group in periode_sample1.groupby('annee'):
    choice = np.random.choice(group.index, size=nb_sample1, replace=False)
    periode_sample2.append(group.loc[choice])
periode_sample2 = pd.concat(periode_sample2)
periode_sample2.set_index('periode', inplace=True)
periode_sample2 = periode_sample2.index
periode_sample2
    

DatetimeIndex(['2018-04-15', '2018-08-19', '2018-07-08', '2018-07-29',
               '2018-05-06', '2018-06-17', '2018-09-02', '2018-07-01',
               '2018-08-05', '2018-08-12', '2018-07-22', '2018-04-08',
               '2020-12-27', '2020-11-29', '2020-10-11', '2020-11-22',
               '2020-12-06', '2020-10-18', '2020-12-13', '2020-11-15',
               '2020-11-08', '2020-12-20', '2020-10-04', '2020-10-25'],
              dtype='datetime64[ns]', name='periode', freq=None)

In [50]:
len(periode_sample2)

24

## Echantillonage des patients

###  1er echantillon des patients

In [51]:
instance_key1 = len(machines_utiles)
patient_sample1 = patients[patients.period.isin(periode_sample2)].copy()
patient_sample1['instance'] = patient_sample1.period.dt.strftime(f'w%Y-%m-%d-m{instance_key1}')
patient_sample1.head()

,number,day,release_date,due_date,duration,period,instance
Date,,,,,,,
2018-04-02,76,1,5,20,5,2018-04-08,w2018-04-08-m21
2018-04-03,67,2,5,20,5,2018-04-08,w2018-04-08-m21
2018-04-04,62,3,5,20,5,2018-04-08,w2018-04-08-m21
2018-04-05,69,4,5,20,5,2018-04-08,w2018-04-08-m21
2018-04-06,72,5,5,20,5,2018-04-08,w2018-04-08-m21


In [52]:
patient_sample1.shape

(144, 7)

### 2eme echantillon des patients

In [53]:
nb_machines2 = 4
instance_key2 = nb_machines2
patient_sample2 = patient_sample1.copy()
patient_sample2['number'] = patient_sample2['number'] * nb_machines2 // nb_machines
patient_sample2['instance'] = patient_sample2.period.dt.strftime(f'w%Y-%m-%d-m{instance_key2}')
patient_sample2.head()

,number,day,release_date,due_date,duration,period,instance
Date,,,,,,,
2018-04-02,14,1,5,20,5,2018-04-08,w2018-04-08-m4
2018-04-03,12,2,5,20,5,2018-04-08,w2018-04-08-m4
2018-04-04,11,3,5,20,5,2018-04-08,w2018-04-08-m4
2018-04-05,13,4,5,20,5,2018-04-08,w2018-04-08-m4
2018-04-06,13,5,5,20,5,2018-04-08,w2018-04-08-m4


In [54]:
patient_sample2.shape

(144, 7)

## Echantillonage des machines

### 1er echantillons des machines

In [55]:
temps_initiaux_hebdo = temps_initiaux.resample('W').min()
temps_initiaux_hebdo = temps_initiaux_hebdo.loc[periode_sample2]
temps_initiaux_hebdo = temps_initiaux_hebdo.unstack()
temps_initiaux_hebdo.index.rename(['code', 'period'], inplace=True)
temps_initiaux_hebdo.name = 'use_duration'
temps_initiaux_hebdo

code      period    
7SXAWU98  2018-04-15       0
          2018-08-19       0
          2018-07-08       0
          2018-07-29       0
          2018-05-06       0
                        ... 
6SX4LV10  2020-11-15    1740
          2020-11-08    1452
          2020-12-20    3204
          2020-10-04      44
          2020-10-25     892
Name: use_duration, Length: 576, dtype: int64

In [56]:
temps_finaux_hebdo = temps_finaux.resample('W').max()
temps_finaux_hebdo = temps_finaux_hebdo.loc[periode_sample2]
temps_finaux_hebdo = temps_finaux_hebdo.unstack()
temps_finaux_hebdo.index.rename(['code', 'period'], inplace=True)
temps_finaux_hebdo.name = 'final_duration'
temps_finaux_hebdo

code      period    
7SXAWU98  2018-04-15       0
          2018-08-19       0
          2018-07-08       0
          2018-07-29       0
          2018-05-06       0
                        ... 
6SX4LV10  2020-11-15    2024
          2020-11-08    1740
          2020-12-20    3512
          2020-10-04     328
          2020-10-25    1172
Name: final_duration, Length: 576, dtype: int64

In [57]:
use_hebdo = use_jours.resample('W').sum()
use_hebdo = use_hebdo.loc[periode_sample2]
use_hebdo = use_hebdo.unstack()
use_hebdo.index.rename(['code', 'period'], inplace=True)
use_hebdo.name = 'nb_jour'
# pannes_hebdo[pannes_hebdo > 0]
use_hebdo

code      period    
7SXAWU98  2018-04-15    0.0
          2018-08-19    0.0
          2018-07-08    0.0
          2018-07-29    0.0
          2018-05-06    0.0
                       ... 
6SX4LV10  2020-11-15    7.0
          2020-11-08    7.0
          2020-12-20    7.0
          2020-10-04    7.0
          2020-10-25    7.0
Name: nb_jour, Length: 576, dtype: float64

In [58]:
pannes_hebdo = pannes.resample('W').sum()
pannes_hebdo = pannes_hebdo.loc[periode_sample2]
pannes_hebdo = pannes_hebdo.unstack()
pannes_hebdo.index.rename(['code', 'period'], inplace=True)
pannes_hebdo.name = 'total_pannes'
pannes_hebdo[pannes_hebdo > 0]

code      period    
7SXAWU90  2020-10-11    3
          2020-10-18    3
7SXAWT98  2020-11-15    3
7SXAWU97  2020-12-27    3
          2020-12-13    3
7SXAWU94  2020-12-06    1
          2020-12-13    2
7SXAWU93  2020-10-18    3
7SXAWU80  2020-12-13    3
7SXAWS80  2020-12-27    3
          2020-12-06    1
          2020-12-13    2
7SXAWU83  2020-12-06    3
          2020-12-13    3
Name: total_pannes, dtype: Int64

In [59]:
machine_fields = [temps_initiaux_hebdo, temps_finaux_hebdo, use_hebdo, pannes_hebdo]
machine_sample1 = pd.concat(machine_fields, axis=1)
machine_sample1.reset_index(inplace=True)
machine_sample1['instance'] = machine_sample1.period.dt.strftime(f'w%Y-%m-%d-m{instance_key1}')
machine_sample1

,code,period,use_duration,final_duration,nb_jour,total_pannes,instance
0,7SXAWU98,2018-04-15,0,0,0.0,0,w2018-04-15-m21
1,7SXAWU98,2018-08-19,0,0,0.0,0,w2018-08-19-m21
2,7SXAWU98,2018-07-08,0,0,0.0,0,w2018-07-08-m21
3,7SXAWU98,2018-07-29,0,0,0.0,0,w2018-07-29-m21
4,7SXAWU98,2018-05-06,0,0,0.0,0,w2018-05-06-m21
...,...,...,...,...,...,...,...
571,6SX4LV10,2020-11-15,1740,2024,7.0,0,w2020-11-15-m21
572,6SX4LV10,2020-11-08,1452,1740,7.0,0,w2020-11-08-m21
573,6SX4LV10,2020-12-20,3204,3512,7.0,0,w2020-12-20-m21
574,6SX4LV10,2020-10-04,44,328,7.0,0,w2020-10-04-m21


In [60]:
machine_sample2 = []
nb_machines1 =len(machines_utiles)
for period, group in machine_sample1.groupby('period'):
    group.sort_values(['total_pannes', 'nb_jour'], 
                      ascending=[True, False], 
                      inplace=True)
    sample = group.iloc[:nb_machines1]
    machine_sample2.append(sample)
    # print(len(sample['code'].to_list()))
machine_sample2 = pd.concat(machine_sample2)
machine_sample2.head()
    

,code,period,use_duration,final_duration,nb_jour,total_pannes,instance
59,7SXAWU81,2018-04-08,320,640,7.0,0,w2018-04-08-m21
83,7SXAWU90,2018-04-08,320,640,7.0,0,w2018-04-08-m21
107,7SXAWS81,2018-04-08,320,640,7.0,0,w2018-04-08-m21
131,7SXAWU96,2018-04-08,320,636,7.0,0,w2018-04-08-m21
155,7SXAWU87,2018-04-08,320,636,7.0,0,w2018-04-08-m21


In [61]:
machine_sample2.shape

(504, 7)

### 2eme echantillon des machines

On va tirer 4 machines aleatoirement pour chaque semaine

In [62]:
machine_sample3 = []
nb_machines2 = 4
for period, group in machine_sample2.groupby('period'):
    group.sort_values(['total_pannes', 'nb_jour'], 
                      ascending=[True, False], 
                      inplace=True)
    choice = np.random.choice(group.index, size=nb_machines2, replace=False)
    print(choice)
    sample = group.loc[choice]
    sample['instance'] = sample.period.dt.strftime(f'w%Y-%m-%d-m{instance_key2}')
    machine_sample3.append(sample)
machine_sample3 = pd.concat(machine_sample3)
machine_sample3.head()

[443 395 371 179]
[ 48 504 456  96]
[484 172  76 100]
[317 221 269 365]
[175 535 271 223]
[554 506 458  74]
[226 538  82 370]
[ 99 507 267 315]
[176 416 152  56]
[513 297 393 249]
[217  49 337 361]
[318 414 486 366]
[ 70 430 382 238]
[566 158 182  14]
[137 233 401 257]
[ 95 407 455 311]
[284 452 164 404]
[ 19 163 115 235]
[ 63 543 327 423]
[469 373 445 565]
[ 64 568 256 184]
[210 498 474 546]
[501 453 549 117]
[156 444 348 396]


,code,period,use_duration,final_duration,nb_jour,total_pannes,instance
443,7SXAWU85,2018-04-08,304,612,7.0,0,w2018-04-08-m4
395,7SXAWU80,2018-04-08,312,624,7.0,0,w2018-04-08-m4
371,7SXAWU91,2018-04-08,312,624,7.0,0,w2018-04-08-m4
179,7SXAWT98,2018-04-08,320,636,7.0,0,w2018-04-08-m4
48,7SXAWU81,2018-04-15,640,968,7.0,0,w2018-04-15-m4


In [63]:
machine_sample3.shape

(96, 7)

## enregistrement des echantillons

In [64]:
patients_path = os.path.join(benchmarkdir, 'echantillons_patients.csv')
patients_sample = pd.concat([patient_sample1, patient_sample2])
patients_sample.to_csv(patients_path)
print(patients_sample.shape)
patients_sample.dtypes

(288, 7)


number                   int64
day                      int64
release_date             int64
due_date                 int64
duration                 int64
period          datetime64[ns]
instance                object
dtype: object

In [65]:
machines_path = os.path.join(benchmarkdir, 'echantillons_machines.csv')
machines_sample = pd.concat([machine_sample2, machine_sample3])
machines_sample.to_csv(machines_path)
print(machines_sample.shape)
machines_sample.dtypes

(600, 7)


code                      object
period            datetime64[ns]
use_duration               int64
final_duration             int64
nb_jour                  float64
total_pannes               Int64
instance                  object
dtype: object